# FAISS 관리

## FAISS 저장시 생기는 파일
### 1. **.faiss 파일**
- **내용**: `.faiss` 파일은 FAISS 인덱스를 저장. 벡터 데이터에 대한 인덱스 구조와 해당 데이터를 검색하기 위한 최적화된 인덱스를 포함.
- **용도**: 인덱스를 다시 로드하여 검색 작업을 수행할 수 있다. 이 파일만 있으면 벡터 데이터에 대한 검색 작업을 다시 수행할 수 있다.

### 2. **.pkl 파일**
- **내용**: `.pkl` 파일은 일반적으로 Python에서 사용하는 피클(pickle) 파일 형식으로 주로 벡터 데이터의 메타정보 또는 인덱스와 관련된 추가 데이터가 저장. 예를 들어, 벡터에 대한 ID, 메타데이터, 인덱스 구성 설정 등이 포함된다.
- **용도**: `.pkl` 파일은 Python 객체를 직렬화하여 저장한 것으로, 인덱스를 다시 로드할 때 사용된다. 이를 통해 검색 과정에서 필요한 부가적인 정보나 설정을 복원할 수 있다.

### 요약
- **.faiss 파일**: 벡터 인덱스 데이터를 저장하는 파일로, 벡터 검색 작업에 직접 사용.
- **.pkl 파일**: Python 객체의 직렬화된 데이터를 저장하는 파일로, 인덱스와 관련된 메타정보나 설정을 포함.

이 파일들은 함께 사용되어 벡터 검색 작업을 빠르게 수행할 수 있도록 한다. 인덱스를 로드할 때 두 파일을 모두 사용하여 완전한 검색 환경을 복원할 수 있다.


## 데이터 저장

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
from app.vectordb import upload_vectordb
from langchain.docstore.document import Document

# upload_vectordb.main(f'{ROOT_PATH}/data/{INDEX_ID}.txt', INDEX_ID)

# 파일 내용 로드 및 처리
text = upload_vectordb.load_and_detect_encoding(f'{ROOT_PATH}/data/{INDEX_ID}.txt')


# 2. 의미별로 chunk로 나누기
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=10)
chunks = text_splitter.split_text(text)

# 각 chunk를 문서 객체로 변환
docs = text_splitter.create_documents(chunks)

# 각 chunk를 문서 객체로 변환 이 과정에서 각 청크를 개별 문서로 다룬다
docs = [Document(page_content=chunk) for chunk in chunks]

# FAISS 인덱스 생성 및 문서 추가
db = FAISS.from_documents(docs, embeddings)

# FAISS 인덱스 저장
db.save_local(f'{ROOT_PATH}/data/vector/{INDEX_ID}')


## 저장된 파일 확인

In [1]:
import pickle
import faiss

# 파일 경로
pkl_file_path = f'{ROOT_PATH}/data/vector/{INDEX_ID}/index'

# .pkl 파일 읽기
with open(f'{pkl_file_path}.pkl', 'rb') as f:
    docstore, uuid_mapping = pickle.load(f)

# 데이터를 출력하여 확인
print(f'# pkl [docstore] = {docstore}')
print(f'# pkl [uuid_mapping][{len(uuid_mapping)}] = {uuid_mapping}')

# .faiss 파일 읽기
index = faiss.read_index(f'{pkl_file_path}.faiss')

# 인덱스 정보 출력
print(f'# 인덱스 정보 출력 = {index}')

# docstore 객체의 메서드와 속성 확인
# print(dir(docstore))

# UUID로 데이터를 조회
for i in uuid_mapping:
    document = docstore.search(uuid_mapping[i])
    print(f'******* {uuid_mapping[i]} = {document}')


NameError: name 'ROOT_PATH' is not defined

## 데이터 조회

In [19]:
from langchain_community.vectorstores import FAISS


# 파일 로드
db = FAISS.load_local(f'{ROOT_PATH}/data/vector/{INDEX_ID}', embeddings, allow_dangerous_deserialization=True)
    
retriever = db.as_retriever(search_kwargs={"k": 3})
query = "보증서특기사항 테이블을 찾아주세요"
# doc = retriever.get_relevant_documents(query)
doc = retriever.invoke(
    input=query,
    config=None,  # 실행 구성을 위해 RunnableConfig 객체를 사용할 수 있음
    search_kwargs={"k": 1}  # 명시적으로 k=1을 전달
)

for d in doc:
    print(d)



page_content='}
### ZGU_GUAS_SPAB_MATR(영업_보증_보증서특기사항){'
page_content='}
### ZGU_GUAS_PSSL_H(영업_보증_보증서_현황_이력){'
page_content='}
### ZGU_GUAS_ISSU_CRDR_DTL(영업_보증_보증서발행_채권자_명세){'


# RAG_SERVER - FAISS 관리

In [1]:
# 경로 추가
import sys
import os

sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), '..')))

from app.utils import get_embedding_model


# 변수 선언
ROOT_PATH = '/Users/passion1014/project/langchain/rag_server'
INDEX_ID = "SAMPLE_simple" # VectorDB INDEX 명
FILE_NAME = f'{INDEX_ID}.txt' # 파일이름

# 임베딩 선언
embeddings = get_embedding_model()

In [1]:
from dotenv import load_dotenv
import os

# 작업디렉토리를 상위경로로 변경
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
os.chdir(parent_dir)

# 환경변수 설정
load_dotenv(dotenv_path=".env.testcase", override=True)

from app.db_model.database import SessionLocal
from app.vectordb.faiss_vectordb import FaissVectorDB


session = SessionLocal()
faissVectorDB = FaissVectorDB(db_session=session, index_name='cg_code_assist')

2025-01-29 18:47:09,952 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-01-29 18:47:09,953 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-01-29 18:47:09,955 INFO sqlalchemy.engine.Engine select current_schema()
2025-01-29 18:47:09,955 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-01-29 18:47:09,957 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-01-29 18:47:09,958 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-01-29 18:47:09,959 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-29 18:47:09,971 INFO sqlalchemy.engine.Engine SELECT faiss_info.id, faiss_info.index_name, faiss_info.index_desc, faiss_info.index_file_path, faiss_info.modified_at, faiss_info.created_at, faiss_info.modified_by, faiss_info.created_by 
FROM faiss_info 
WHERE faiss_info.index_name = %(index_name_1)s
2025-01-29 18:47:09,972 INFO sqlalchemy.engine.Engine [generated in 0.00082s] {'index_name_1': 'cg_code_assist'}


RuntimeError: Error in faiss::FileIOReader::FileIOReader(const char *) at /Users/runner/work/faiss-wheels/faiss-wheels/faiss/faiss/impl/io.cpp:68: Error: 'f' failed: could not open data/vector/cg_code_assist.index for reading: No such file or directory

In [ ]:
# 전체 문서 확인(by faiss_index)
faissVectorDB.get_all_documents()

In [ ]:
# 검색
faissVectorDB.search_similar_documents("getPigVital(Long vitalCode") # 검색어로 조회